In [1]:
import numpy as np
import joblib
import pandas as pd

# Ruta a los archivos
X_path = "../data_subsets/train_test/X_train_50.npy"
scaler_path = "../models/standard_scaler.pkl"

# Cargar scaler y datos
X_train = np.load(X_path)
scaler = joblib.load(scaler_path)

# Columnas originales (en el mismo orden del fit del scaler)
column_names = [
    'segment_num', 'airline_code', 'aircraft_type', 'duration_seconds',
    'distance_miles', 'departure_hour', 'departure_weekday', 'cabin',
    'arr_airport', 'dep_airport', 'startingAirport', 'destinationAirport',
    'fareBasisCode', 'isBasicEconomy', 'isRefundable', 'isNonStop',
    'baseFare', 'seatsRemaining', 'elapsedDays', 'days_in_advance',
    'totalTravelDistance'
]

# Crear DataFrame con columnas
df = pd.DataFrame(X_train, columns=column_names)

# Ver algunas filas
print(df.head())

# Ver resumen estadístico
print(df.describe())


   segment_num  airline_code  aircraft_type  duration_seconds  distance_miles  \
0     0.915816      1.202734      -0.178342         -0.979411       -1.382997   
1    -0.795679     -0.336449       0.907688         -0.634388       -0.459714   
2    -0.795679      1.642501      -0.178342          0.254709        0.118123   
3    -0.795679     -0.776215      -0.178342          0.281249        0.234318   
4    -0.795679     -0.776215       0.093166          0.626272        0.488692   

   departure_hour  departure_weekday     cabin  arr_airport  dep_airport  ...  \
0       -0.617396           0.085090 -0.020643     0.493967     0.495845  ...   
1        0.740004          -1.459735 -0.020643     1.692334     1.008306  ...   
2       -0.423482           0.600032 -0.020643     1.692334     0.495845  ...   
3        0.546090           0.085090 -0.020643    -1.389182    -1.041536  ...   
4        0.546090          -1.459735 -0.020643     1.007553     0.666666  ...   

   destinationAirport  far

In [2]:
import pandas as pd
import glob

# Ruta a los chunks originales con los valores codificados
chunk_files = sorted(glob.glob("../data_subsets/final_dataset_chunks/chunk_*.parquet"))

# Leer solo el primer chunk (o varios si quieres juntar más)
df = pd.read_parquet(chunk_files[0])

print(f"✅ Chunk cargado con {df.shape[0]} filas y {df.shape[1]} columnas")

# Seleccionar las columnas categóricas codificadas numéricamente
categorical_columns = [
    "airline_code", "aircraft_type", "cabin", "arr_airport", "dep_airport",
    "startingAirport", "destinationAirport", "fareBasisCode"
]

# Mostrar cantidad de valores únicos por columna
print("\n🔍 Número de valores únicos por columna categórica:")
for col in categorical_columns:
    uniques = df[col].nunique()
    print(f"• {col}: {uniques} valores únicos")

# Opcional: ver qué valores numéricos tiene cada columna
print("\n🧾 Valores codificados por columna (ejemplo):")
for col in categorical_columns:
    print(f"\n➡️ {col}:\n{df[col].value_counts().sort_index()}")


✅ Chunk cargado con 1048576 filas y 23 columnas

🔍 Número de valores únicos por columna categórica:
• airline_code: 11 valores únicos
• aircraft_type: 9 valores únicos
• cabin: 4 valores únicos
• arr_airport: 19 valores únicos
• dep_airport: 19 valores únicos
• startingAirport: 16 valores únicos
• destinationAirport: 16 valores únicos
• fareBasisCode: 5851 valores únicos

🧾 Valores codificados por columna (ejemplo):

➡️ airline_code:
airline_code
2        952
4       6287
5     350313
6      77071
7     257876
8      18229
11      1126
12       220
14     80698
15      1842
16    253962
Name: count, dtype: int64

➡️ aircraft_type:
aircraft_type
0.0      56011
17.0    400722
18.0    355197
19.0     44730
20.0       219
21.0    172034
22.0       325
32.0       952
36.0      3934
Name: count, dtype: int64

➡️ cabin:
cabin
3         121
4     1046278
6        1288
12        889
Name: count, dtype: int64

➡️ arr_airport:
arr_airport
1      87128
2      63159
3      74646
4     173157
5     

In [5]:
import pandas as pd
import glob
import json
import numpy as np

# Ruta a los chunks codificados
chunk_files = sorted(glob.glob("../data_subsets/final_dataset_chunks/chunk_*.parquet"))

# Columnas codificadas (categóricas y numéricas)
categorical_and_numerical_columns = [
    "segment_num", "airline_code", "aircraft_type", "cabin",
    "arr_airport", "dep_airport", "startingAirport", "destinationAirport",
    "fareBasisCode", "duration_seconds", "distance_miles", "departure_hour",
    "departure_weekday", "isBasicEconomy", "isRefundable", "isNonStop",
    "baseFare", "seatsRemaining", "elapsedDays", "days_in_advance",
    "totalTravelDistance"
]

# Diccionario para acumular valores únicos
unique_values = {col: set() for col in categorical_and_numerical_columns}

print(f"🔄 Procesando {len(chunk_files)} chunks...")

for path in chunk_files:
    df = pd.read_parquet(path)
    for col in categorical_and_numerical_columns:
        if col in df.columns:
            unique_values[col].update(df[col].dropna().unique())

# Convertir los valores a tipos compatibles con JSON
final_values = {}
for col, values in unique_values.items():
    cleaned = []
    for v in values:
        if isinstance(v, (np.integer, np.int32, np.int64)):
            cleaned.append(int(v))
        elif isinstance(v, (np.floating, np.float32, np.float64)):
            cleaned.append(float(v))
        else:
            cleaned.append(v)
    final_values[col] = sorted(cleaned)

# Asegurar que todas las columnas estén presentes
for col in categorical_and_numerical_columns:
    final_values.setdefault(col, [])

# Guardar en archivo JSON en carpeta data_subsets
with open("../data_subsets/column_values.json", "w") as f:
    json.dump(final_values, f, indent=2)

print("✅ Valores únicos guardados en '../data_subsets/column_values.json'")

🔄 Procesando 56 chunks...
✅ Valores únicos guardados en '../data_subsets/column_values.json'
